In [1]:
import torch 
import torch.nn as nn
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt

# Multilayer Perceptron

Let's start with the most simple neural network out there: the Multilayer Perceptron. This neural network model just consists on sequential multiple linear transformations with some activation functions in between. These activation functions are the ones that gave the non-linear properties to the model.

In [2]:
"""
Defining the dataset

In this case we are using the simple digits dataset, which consists on 8x8 gray scaled images of written numbers 0-9 in a very low r
"""
class DigitsDataset:
    def __init__(self):
        digits_data_ = load_digits()
        self.imgs = digits_data_.images.reshape((-1,1,8,8)) / 16.0
        self.target = digits_data_.target
    def __getitem__(self,idx):
        return torch.from_numpy(self.imgs[idx]).float(), self.target[idx]
    def __len__(self):
        return len(self.imgs)

In [3]:
# Creating the digits dataset object
digits_ds = DigitsDataset()

In [4]:
# Splitting the dataset into train and test dataset
N = len(digits_ds)
train_size = int(0.8*N)
test_size = N - train_size
train_ds, test_ds = torch.utils.data.random_split(digits_ds,[train_size, test_size])

In [5]:
# Creating the datataloaders
train_dl = torch.utils.data.DataLoader(train_ds,batch_size=32)
test_dl = torch.utils.data.DataLoader(test_ds,batch_size=32)

In [6]:
# Creating a simple MLP
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(64,32),
    nn.ReLU(inplace=True),
    nn.Linear(32,10)
)
"""
To train the model we need to measure how well the model is doing with respect the dataset. 
To do so, we use a loss function proper for the task that is being handled.

In this case, the task is image classification and thus the most proper loss function is the Cross Entropy Loss
"""
crit = nn.CrossEntropyLoss()

"""
Once we calculate the loss and performa the backpropagation to calculate the gradient of the loss w.r.t. 
each parameter, we need a way to update the parameters using the gradient. This is called optimization. 
The simplest optimization method there is, it is the SGD. Which just follows the negative of the gradient scaled by
a factor called learning rate.
"""
optim = torch.optim.SGD(model.parameters(),lr=0.1)

In [7]:
# Training the model
for epoch in range(10):
    tot_loss = 0
    for x,y in train_dl:
        # Set the gradients of all parameters to zero 
        # Since the when we perform backpropagation, the gradients accumulates
        optim.zero_grad()
        # Forward the current input
        o = model(x)
        # Calculate the loss
        l = crit(o,y)
        tot_loss += l.item()
        # Backpropagate
        l.backward()
        # Update the parameters using the chosen optimizer
        optim.step()
    print(tot_loss/len(train_dl))

2.1936256567637127
1.7785868936114841
1.1796074668566385
0.7578269918759664
0.5424757904476589
0.42476282748911115
0.3521336055464215
0.30306824247042335
0.26759569727712207
0.24069624212053087


# Convolutional Neural Network

The operation of convolution helps to extract spatial features from the input data.

In [8]:
"""
Defining the model
The convolutional layers (along with the activation functions and pooling layers) are in charge of extract 
features from the input, while the last layer (linear) is in charge of classify those features into the 
available classes
"""
model = nn.Sequential(
    nn.Conv2d(1,16,3,padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(16,32,3,padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(32,32,3,padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.Linear(32,10)
)
# Since this is still a classfication problem we are using Cross Entropy Loss as our loss function
crit = nn.CrossEntropyLoss()
# We are also still using the SGD optimizer
optim = torch.optim.SGD(model.parameters(),lr=0.1)

In [9]:
# Traininng process: same as before
for epoch in range(10):
    tot_loss = 0
    for x,y in train_dl:
        optim.zero_grad()
        o = model(x)
        l = crit(o,y)
        tot_loss += l.item()
        l.backward()
        optim.step()
    print(tot_loss/len(train_dl))

2.2970400121476917
2.271151383717855
2.192166386710273
1.8586749023861355
1.2125170522265964
0.7476721353001065
0.48960680332448747
0.3369720273547702
0.24317965557177862
0.1877035178244114


# Skip connections and Residual Networks

When training deep neural networks, we face a problem called vanishing gradient problem. This problem is due to the fact that when backpropagating the gradient (in other words applying the chain rule) the gradient becomes so small for the deeper layers that its training becomes extremely slow and unstable. Thus, in order to prevent this we use skip connections (connecting layers that are not consecutive) so the gradient has an alternative path where it can skip some layers and avoid being diminished. 

In [10]:
# Defining the basic residual block
class BasicResidualBlock(nn.Module):
    def __init__(self, in_planes, out_planes):
        super().__init__()
        self.direct = nn.Sequential(
            nn.Conv2d(in_planes, out_planes,3,padding=1),
            nn.BatchNorm2d(out_planes),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_planes,out_planes,1),
            nn.BatchNorm2d(out_planes)
        )
        
        self.shortcut = nn.Sequential()
        if in_planes != out_planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, out_planes,1),
                nn.BatchNorm2d(out_planes)
            )
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        o = self.direct(x)
        return self.relu(o + self.shortcut(x))

In [11]:
model = nn.Sequential(
    BasicResidualBlock(1,16),
    nn.MaxPool2d(kernel_size=2),
    BasicResidualBlock(16,32),
    nn.MaxPool2d(kernel_size=2),
    BasicResidualBlock(32,32),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.Linear(32,10)
)
crit = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(),lr=0.1)

In [12]:
for epoch in range(10):
    tot_loss = 0
    for x,y in train_dl:
        optim.zero_grad()
        o = model(x)
        l = crit(o,y)
        tot_loss += l.item()
        l.backward()
        optim.step()
    print(tot_loss/len(train_dl))

1.5384304831425348
0.13391249655849405
0.03679622666289409
0.017365182387746043
0.009804410176972549
0.006703468525989188
0.00521805355221861
0.004274974189077814
0.0036218349407944415
0.003133683400746021


# Autoencoder: Linear and Deconvolution

In [13]:
class Reshape(nn.Module):
    def __init__(self, new_shape):
        super().__init__()
        self.new_shape = new_shape
        
    def forward(self, x):
        return x.view(self.new_shape)

class LinearAutoencoder(nn.Module):
    def __init__(self, latent=16,img_shape=(1,8,8)):
        super().__init__()
        
        s = 1
        for elem in img_shape:
            s *= elem
        
        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(s,32),
            nn.ReLU(inplace=True),
            nn.Linear(32,latent),
            nn.Tanh()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(latent,32),
            nn.ReLU(inplace=True),
            nn.Linear(32,s),
            nn.Sigmoid(),
            Reshape((-1,*img_shape))
        )
    
    def forward(self,x):
        enc = self.encoder(x)
        return self.decoder(enc)

In [14]:
model = LinearAutoencoder()
crit = nn.MSELoss()
optim = torch.optim.SGD(model.parameters(),lr=0.4)

In [15]:
model.train()
for epoch in range(20):
    tot_loss = 0
    for x,_ in train_dl:
        optim.zero_grad()
        o = model(x)
        l = crit(o,x)
        tot_loss += l.item()
        l.backward()
        optim.step()
    print(tot_loss/len(train_dl))

0.17266242636574639
0.15382812983459895
0.11695603628953298
0.08520220037963655
0.07660132348537445
0.07474049031734467
0.07413673351208369
0.0738513751162423
0.07368429899215698
0.07357371366686291
0.07349442889293036
0.07343419343233108
0.07338638156652451
0.07334708107842339
0.07331384966770808
0.07328506377008226
0.07325959354639053
0.07323662655221091
0.07321557253599167
0.07319597933027479


In [16]:
model.eval()
fig, axs = plt.subplots(2,4)
for x,_ in test_dl:
    with torch.no_grad():
        o = model(x).numpy()
    for i in range(4):
        axs[0][i].imshow(x[i][0])
        axs[1][i].imshow(o[i][0])
    break

In [17]:
class ConvDeconvAutoencoder(nn.Module):
    def __init__(self, latent=16,img_shape=(1,8,8)):
        super().__init__()
        
        s= 1
        for elem in img_shape:
            s *= elem
        
        self.encoder = nn.Sequential(
            nn.Conv2d(img_shape[0],8,3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(8,16,3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(16,latent,3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
            nn.Tanh()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(latent,32),
            nn.ReLU(inplace=True),
            Reshape((-1,8,2,2)),
            nn.ConvTranspose2d(8,4,2,stride=2),
            nn.BatchNorm2d(4),
            nn.ConvTranspose2d(4,1,2,stride=2),
            nn.Sigmoid()
        )
    
    def forward(self,x):
        enc = self.encoder(x)
        return self.decoder(enc)

In [18]:
model = ConvDeconvAutoencoder()
crit = nn.MSELoss()
optim = torch.optim.SGD(model.parameters(),lr=0.4)

In [19]:
model.train()
for epoch in range(20):
    tot_loss = 0
    for x,_ in train_dl:
        optim.zero_grad()
        o = model(x)
        l = crit(o,x)
        tot_loss += l.item()
        l.backward()
        optim.step()
    print(tot_loss/len(train_dl))

0.14662512905067868
0.08730662730005052
0.07875098503298229
0.07584432860215505
0.07248887966076532
0.06893152445554733
0.06472136593527264
0.059508122917678624
0.05596679018603431
0.05298583151565658
0.050667735603120594
0.0486472160451942
0.046542392505539786
0.044687312179141576
0.04326356119579739
0.04209235351946619
0.0409843525952763
0.03926034627689256
0.03880033103956117
0.03725074844227897


In [20]:
model.eval()
fig, axs = plt.subplots(2,4)
for x,_ in test_dl:
    with torch.no_grad():
        o = model(x).numpy()
    for i in range(4):
        axs[0][i].imshow(x[i][0])
        axs[1][i].imshow(o[i][0])
    break

# GAN: Linear and Deconvolution

In [21]:
latent = 16
model_generator = nn.Sequential(
    nn.Linear(latent,32),
    nn.ReLU(),
    nn.Linear(32,64),
    nn.Sigmoid(),
    Reshape((-1,1,8,8))
)

model_discriminator = nn.Sequential(
    nn.Conv2d(1,16,3,padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(16,32,3,padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(32,32,3,padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.Linear(32,1)
)

crit = nn.BCEWithLogitsLoss()
optim_generator = torch.optim.SGD(model_generator.parameters(),lr=0.2)
optim_discriminator = torch.optim.SGD(model_discriminator.parameters(),lr=0.02)

In [27]:
model_generator.train()
model_discriminator.train()
for epoch in range(10):
    tot_loss_discriminator = 0
    tot_loss_generator = 0
    for x,y in train_dl:
        optim_generator.zero_grad()
        optim_discriminator.zero_grad()
        
        N = len(x)
        real_labels = torch.ones((N,1))
        fake_labels = torch.zeros((N,1))
        
        noise = torch.randn((N,latent))
        fake_imgs = model_generator(noise).detach()
        loss_discriminator = 0.5 * (crit(model_discriminator(fake_imgs),fake_labels) + \
                            crit(model_discriminator(x),real_labels))
        
        noise = torch.randn((N,latent))
        fake_imgs = model_generator(noise)
        loss_generator = crit(model_discriminator(fake_imgs),real_labels)
        
        loss_discriminator.backward()
        loss_generator.backward()
        
        tot_loss_discriminator += loss_discriminator.item()
        tot_loss_generator += loss_generator.item()
        
        optim_generator.step()
        optim_discriminator.step()
    print(f"loss discriminattor: {tot_loss_discriminator / len(train_dl)}, loss generator: {tot_loss_generator / len(train_dl)}")

loss discriminattor: 0.6460381507873535, loss generator: 0.36079277396202086
loss discriminattor: 0.6673621535301208, loss generator: 0.3616842372549905
loss discriminattor: 0.6948077956835429, loss generator: 0.35605476796627045
loss discriminattor: 0.7628933270772298, loss generator: 0.3280831025706397
loss discriminattor: 0.7327257951100667, loss generator: 0.3340194331275092
loss discriminattor: 0.7890731877750821, loss generator: 0.3068840616279178
loss discriminattor: 0.7572043220202128, loss generator: 0.31687471204333834
loss discriminattor: 0.7146069407463074, loss generator: 0.3315608885553148
loss discriminattor: 0.7106145130263435, loss generator: 0.3315004732873705
loss discriminattor: 0.6969422062238058, loss generator: 0.33694734970728557


In [28]:
model_generator.eval()
fig, axs = plt.subplots(2,4)

noise = torch.randn((8,latent))
with torch.no_grad():
    o = model_generator(noise).numpy()
    
for i in range(8):
    axs[i//4, i%4].imshow(o[i][0])


In [29]:
tmp = nn.ConvTranspose2d(1,3,3)
tmp(torch.zeros((1,1,2,2))).shape

torch.Size([1, 3, 4, 4])

In [30]:
latent = 16
model_generator = nn.Sequential(
    nn.Linear(latent,64),
    nn.ReLU(inplace=True),
    Reshape((-1,64,1,1)),
    nn.ConvTranspose2d(64,16,2),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(16,8,3),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(8,1,5),
    nn.Sigmoid()
)

model_discriminator = nn.Sequential(
    nn.Conv2d(1,16,3,padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(16,32,3,padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(32,32,3,padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.Linear(32,1)
)

crit = nn.BCEWithLogitsLoss()
optim_generator = torch.optim.SGD(model_generator.parameters(),lr=0.1)
optim_discriminator = torch.optim.SGD(model_discriminator.parameters(),lr=0.001)

In [33]:
model_generator.train()
model_discriminator.train()
for epoch in range(10):
    tot_loss_discriminator = 0
    tot_loss_generator = 0
    for x,y in train_dl:
        optim_generator.zero_grad()
        optim_discriminator.zero_grad()
        
        N = len(x)
        real_labels = torch.ones((N,1))
        fake_labels = torch.zeros((N,1))
        
        noise = torch.randn((N,latent))
        fake_imgs = model_generator(noise).detach()
        loss_discriminator = 0.5 * (crit(model_discriminator(fake_imgs),fake_labels) + \
                            crit(model_discriminator(x),real_labels))
        loss_discriminator.backward()
        optim_discriminator.step()
        
        noise = torch.randn((N,latent))
        fake_imgs = model_generator(noise)
        loss_generator = crit(model_discriminator(fake_imgs),real_labels)
        loss_generator.backward()
        optim_generator.step()
        
        tot_loss_discriminator += loss_discriminator.item()
        tot_loss_generator += loss_generator.item()
        
    print(f"loss discriminattor: {tot_loss_discriminator / len(train_dl)}, loss generator: {tot_loss_generator / len(train_dl)}")

loss discriminattor: 0.6880115257369147, loss generator: 0.683040992418925
loss discriminattor: 0.6876822193463643, loss generator: 0.6827542728847927
loss discriminattor: 0.687336430284712, loss generator: 0.6825026949246724
loss discriminattor: 0.6869725333319769, loss generator: 0.682241087489658
loss discriminattor: 0.6866139875517951, loss generator: 0.6819555229610867
loss discriminattor: 0.6862587107552423, loss generator: 0.6816265967157152
loss discriminattor: 0.6859024657143487, loss generator: 0.6812769995795356
loss discriminattor: 0.6855558461613125, loss generator: 0.6809089581171671
loss discriminattor: 0.6852169659402635, loss generator: 0.6805261770884196
loss discriminattor: 0.6848680893580119, loss generator: 0.6801503446367052


tensor(-1.8657)


In [35]:
model_generator.eval()
fig, axs = plt.subplots(2,4)

noise = torch.randn((8,latent))
print(noise[0][0])
with torch.no_grad():
    o = model_generator(noise).numpy()
    
for i in range(8):
    axs[i//4, i%4].imshow(o[i][0])

